### Partitioner

- When processing, Spark assigns one task for each partition and each worker threads can only process one task at a time. Thus, with too few partitions, the application won’t utilize all the cores available in the cluster and it can cause data skewing problem; with too many partitions, it will bring overhead for Spark to manage too many small tasks.

- Partitioner class is used to partition data based on keys. It accepts two parameters numPartitions and partitionFunc to initiate as the following code shows:

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.rdd import portable_hash
from pyspark import Row

appName = "PySpark Partition Example"
master = "local[8]"

# Create Spark session with Hive supported.
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .getOrCreate()
print(spark.version)
spark.sparkContext.setLogLevel("ERROR")

2.4.4


There are 12 records populated:

In [2]:
# Populate sample data
countries = ("CN", "AU", "US")
data = []
for i in range(1, 13):
    data.append({"ID": i, "Country": countries[i % 3],  "Amount": 10+i})
df = spark.createDataFrame(data)
df.show()

/Users/erikapat/anaconda3/lib/python3.7/site-packages/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+------+-------+---+
|Amount|Country| ID|
+------+-------+---+
|    11|     AU|  1|
|    12|     US|  2|
|    13|     CN|  3|
|    14|     AU|  4|
|    15|     US|  5|
|    16|     CN|  6|
|    17|     AU|  7|
|    18|     US|  8|
|    19|     CN|  9|
|    20|     AU| 10|
|    21|     US| 11|
|    22|     CN| 12|
+------+-------+---+



The output from print_partitions function is shown below:

In [3]:
def print_partitions(df):
    numPartitions = df.rdd.getNumPartitions()
    print("Total partitions: {}".format(numPartitions))
    print("Partitioner: {}".format(df.rdd.partitioner))
    df.explain()
    parts = df.rdd.glom().collect()
    i = 0
    j = 0
    for p in parts:
        print("Partition {}:".format(i))
        for r in p:
            print("Row {}:{}".format(j, r))
            j = j+1
        i = i+1


In [4]:
print_partitions(df)

Total partitions: 8
Partitioner: None
== Physical Plan ==
Scan ExistingRDD[Amount#0L,Country#1,ID#2L]
Partition 0:
Row 0:Row(Amount=11, Country='AU', ID=1)
Partition 1:
Row 1:Row(Amount=12, Country='US', ID=2)
Row 2:Row(Amount=13, Country='CN', ID=3)
Partition 2:
Row 3:Row(Amount=14, Country='AU', ID=4)
Partition 3:
Row 4:Row(Amount=15, Country='US', ID=5)
Row 5:Row(Amount=16, Country='CN', ID=6)
Partition 4:
Row 6:Row(Amount=17, Country='AU', ID=7)
Partition 5:
Row 7:Row(Amount=18, Country='US', ID=8)
Row 8:Row(Amount=19, Country='CN', ID=9)
Partition 6:
Row 9:Row(Amount=20, Country='AU', ID=10)
Partition 7:
Row 10:Row(Amount=21, Country='US', ID=11)
Row 11:Row(Amount=22, Country='CN', ID=12)


### Repartition data
Let’s repartition the data to three partitions only by Country column.

In [5]:
numPartitions = 3

df = df.repartition(numPartitions, "Country")

print_partitions(df)

Total partitions: 3
Partitioner: None
== Physical Plan ==
Exchange hashpartitioning(Country#1, 3)
+- Scan ExistingRDD[Amount#0L,Country#1,ID#2L]
Partition 0:
Partition 1:
Row 0:Row(Amount=12, Country='US', ID=2)
Row 1:Row(Amount=13, Country='CN', ID=3)
Row 2:Row(Amount=15, Country='US', ID=5)
Row 3:Row(Amount=16, Country='CN', ID=6)
Row 4:Row(Amount=18, Country='US', ID=8)
Row 5:Row(Amount=19, Country='CN', ID=9)
Row 6:Row(Amount=21, Country='US', ID=11)
Row 7:Row(Amount=22, Country='CN', ID=12)
Partition 2:
Row 8:Row(Amount=11, Country='AU', ID=1)
Row 9:Row(Amount=14, Country='AU', ID=4)
Row 10:Row(Amount=17, Country='AU', ID=7)
Row 11:Row(Amount=20, Country='AU', ID=10)


You may expect that each partition includes data for each Country but that is not the case. Why? Because repartition function by default uses hash partitioning. For different country code, it may be allocated into the same partition number.

We can verify this by using the following code to calculate the hash

In [7]:
udf_portable_hash = udf(lambda str: portable_hash(str))

df = df.withColumn("Hash#", udf_portable_hash(df.Country))

df = df.withColumn("Partition#", df["Hash#"] % numPartitions)

df.show()

+------+-------+---+--------------------+----------+
|Amount|Country| ID|               Hash#|Partition#|
+------+-------+---+--------------------+----------+
|    12|     US|  2|-8328537658613580243|      -1.0|
|    13|     CN|  3|-7458853143580063552|      -1.0|
|    15|     US|  5|-8328537658613580243|      -1.0|
|    16|     CN|  6|-7458853143580063552|      -1.0|
|    18|     US|  8|-8328537658613580243|      -1.0|
|    19|     CN|  9|-7458853143580063552|      -1.0|
|    21|     US| 11|-8328537658613580243|      -1.0|
|    22|     CN| 12|-7458853143580063552|      -1.0|
|    11|     AU|  1| 6593628092971972691|       0.0|
|    14|     AU|  4| 6593628092971972691|       0.0|
|    17|     AU|  7| 6593628092971972691|       0.0|
|    20|     AU| 10| 6593628092971972691|       0.0|
+------+-------+---+--------------------+----------+



This output is consistent with the previous one as record ID 1,4,7,10 are allocated to one partition while the others are allocated to another question. There is also one partition with empty content as no records are allocated to that partition.

### Allocate one partition for each key value
For the above example, if we want to allocate one partition for each Country (CN, US, AU), what should we do?

Well, the first thing we can try is to increase the partition number. In this way, the chance for allocating each different value to different partition is higher.

So if we increate the partition number to 5.

The output shows that each country’s data is now located in the same partition:

In [8]:
numPartitions = 5

df = df.repartition(numPartitions, "Country")

print_partitions(df)

udf_portable_hash = udf(lambda str: portable_hash(str))

df = df.withColumn("Hash#", udf_portable_hash(df.Country))

df = df.withColumn("Partition#", df["Hash#"] % numPartitions)

df.show()

Total partitions: 5
Partitioner: None
== Physical Plan ==
Exchange hashpartitioning(Country#1, 5)
+- *(1) Project [Amount#0L, Country#1, ID#2L, pythonUDF1#47 AS Hash##17, (cast(pythonUDF1#47 as double) % 3.0) AS Partition##22]
   +- BatchEvalPython [<lambda>(Country#1), <lambda>(Country#1)], [Amount#0L, Country#1, ID#2L, pythonUDF0#46, pythonUDF1#47]
      +- Exchange hashpartitioning(Country#1, 3)
         +- Scan ExistingRDD[Amount#0L,Country#1,ID#2L]
Partition 0:
Partition 1:
Partition 2:
Row 0:Row(Amount=12, Country='US', ID=2, Hash#='-8328537658613580243', Partition#=-1.0)
Row 1:Row(Amount=15, Country='US', ID=5, Hash#='-8328537658613580243', Partition#=-1.0)
Row 2:Row(Amount=18, Country='US', ID=8, Hash#='-8328537658613580243', Partition#=-1.0)
Row 3:Row(Amount=21, Country='US', ID=11, Hash#='-8328537658613580243', Partition#=-1.0)
Partition 3:
Row 4:Row(Amount=13, Country='CN', ID=3, Hash#='-7458853143580063552', Partition#=-1.0)
Row 5:Row(Amount=16, Country='CN', ID=6, Hash#='-

However, what if the hashing algorithm generates the same hash code/number?

#### Use partitionBy function
To address the above issue, we can create a customised partitioning function.

In [13]:
def country_partitioning(k):
    return countries.index(k)
    
udf_country_hash = udf(lambda str: country_partitioning(str))

In [14]:
df = df.rdd \
    .map(lambda el: (el["Country"], el)) \
    .partitionBy(numPartitions, country_partitioning) \
    .toDF()
print_partitions(df)

df = df.withColumn("Hash#", udf_country_hash(df[0]))
df = df.withColumn("Partition#", df["Hash#"] % numPartitions)
df.show(100, False)

Total partitions: 3
Partitioner: None
== Physical Plan ==
Scan ExistingRDD[_1#19,_2#20]
Partition 0:
Row 0:Row(_1='CN', _2=Row(Amount=13, Country='CN', ID=3))
Row 1:Row(_1='CN', _2=Row(Amount=16, Country='CN', ID=6))
Row 2:Row(_1='CN', _2=Row(Amount=19, Country='CN', ID=9))
Row 3:Row(_1='CN', _2=Row(Amount=22, Country='CN', ID=12))
Partition 1:
Row 4:Row(_1='AU', _2=Row(Amount=11, Country='AU', ID=1))
Row 5:Row(_1='AU', _2=Row(Amount=14, Country='AU', ID=4))
Row 6:Row(_1='AU', _2=Row(Amount=17, Country='AU', ID=7))
Row 7:Row(_1='AU', _2=Row(Amount=20, Country='AU', ID=10))
Partition 2:
Row 8:Row(_1='US', _2=Row(Amount=12, Country='US', ID=2))
Row 9:Row(_1='US', _2=Row(Amount=15, Country='US', ID=5))
Row 10:Row(_1='US', _2=Row(Amount=18, Country='US', ID=8))
Row 11:Row(_1='US', _2=Row(Amount=21, Country='US', ID=11))
+---+------------+-----+----------+
|_1 |_2          |Hash#|Partition#|
+---+------------+-----+----------+
|CN |[13, CN, 3] |0    |0.0       |
|CN |[16, CN, 6] |0    |0.0 

In [11]:
print_partitions(df)

df.write.mode("overwrite").partitionBy("Country").csv("data/example2.csv", header=True)

Total partitions: 3
Partitioner: None
== Physical Plan ==
Exchange hashpartitioning(Country#1, 3)
+- Scan ExistingRDD[Amount#0L,Country#1,ID#2L]
Partition 0:
Partition 1:
Row 0:Row(Amount=12, Country='US', ID=2)
Row 1:Row(Amount=13, Country='CN', ID=3)
Row 2:Row(Amount=15, Country='US', ID=5)
Row 3:Row(Amount=16, Country='CN', ID=6)
Row 4:Row(Amount=18, Country='US', ID=8)
Row 5:Row(Amount=19, Country='CN', ID=9)
Row 6:Row(Amount=21, Country='US', ID=11)
Row 7:Row(Amount=22, Country='CN', ID=12)
Partition 2:
Row 8:Row(Amount=11, Country='AU', ID=1)
Row 9:Row(Amount=14, Country='AU', ID=4)
Row 10:Row(Amount=17, Country='AU', ID=7)
Row 11:Row(Amount=20, Country='AU', ID=10)


Through this customised partitioning function, we guarantee each different country code gets a unique deterministic hash number.

Now if we change the number of partitions to 2, both US and CN records will be allocated to one partition because:

- CN (Hash# = 0): 0%2 = 0
- US (Hash# = 2): 2%2 = 0
- AU (Hash# = 1): 1%2 = 1

## group by

In [30]:
df = df.repartition("Country")
print(df.rdd.getNumPartitions())
df.write.mode("overwrite").csv("data/example.csv", header=True)

200


### OTHER EXAMPLES

In [18]:
transactions = [
    {'name': 'Bob', 'amount': 100, 'country': 'United Kingdom'},
    {'name': 'James', 'amount': 15, 'country': 'United Kingdom'},
    {'name': 'Marek', 'amount': 51, 'country': 'Poland'},
    {'name': 'Johannes', 'amount': 200, 'country': 'Germany'},
    {'name': 'Paul', 'amount': 75, 'country': 'Poland'},
]

In [20]:
rdd = spark.sparkContext \
        .parallelize(transactions) \
        .map(lambda x: Row(**x))
    
df = spark.createDataFrame(rdd)
df.show()

+------+--------------+--------+
|amount|       country|    name|
+------+--------------+--------+
|   100|United Kingdom|     Bob|
|    15|United Kingdom|   James|
|    51|        Poland|   Marek|
|   200|       Germany|Johannes|
|    75|        Poland|    Paul|
+------+--------------+--------+



In [21]:
print("Number of partitions: {}".format(df.rdd.getNumPartitions()))
print("Partitioner: {}".format(rdd.partitioner))
print("Partitions structure: {}".format(df.rdd.glom().collect()))
    

Number of partitions: 8
Partitioner: None
Partitions structure: [[], [Row(amount=100, country='United Kingdom', name='Bob')], [], [Row(amount=15, country='United Kingdom', name='James')], [Row(amount=51, country='Poland', name='Marek')], [], [Row(amount=200, country='Germany', name='Johannes')], [Row(amount=75, country='Poland', name='Paul')]]


In [22]:
# Repartition by column
df2 = df.repartition("country")
    
print("\nAfter 'repartition()'")
print("Number of partitions: {}".format(df2.rdd.getNumPartitions()))
print("Partitioner: {}".format(df2.rdd.partitioner))
print("Partitions structure: {}".format(df2.rdd.glom().collect()))


After 'repartition()'
Number of partitions: 200
Partitioner: None
Partitions structure: [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [Row(amount=200, country='Germany', name='Johannes')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [Row(amount=51, country='Poland', name='Marek'), Row(amount=75, country='Poland', name='Paul')], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

## Conclusion:
- When your are deciding the number of partitions to use, you need to take access paths into consideration, for example, are your partition keys commonly used in filters?

- But keep in mind that partitioning will not be helpful in all applications. For example, if a given data frame is used only once, there is no point in partitioning it in advance. It’s useful only when a dataset is reused multiple times (in key-oriented situations using functions like join()).

- Operations that benefit from partitioning
All operations performing shuffling data by key will benefit from partitioning. Some examples are cogroup(), groupWith(), join(), leftOuterJoin(), rightOuterJoin(), groupByKey(), reduceByKey(), combineByKey() or lookup().

## References

- Data Partitioning in Spark (PySpark) In-depth Walkthrough https://kontext.tech/column/spark/296/data-partitioning-in-spark-pyspark-in-depth-walkthrough
- Data Partitioning Functions in Spark (PySpark) Deep Dive. https://kontext.tech/column/spark/299/data-partitioning-functions-in-spark-pyspark-explained
- http://ethen8181.github.io/machine-learning/big_data/spark_partitions.html
- https://towardsdatascience.com/the-art-of-joining-in-spark-dcbd33d693c
- Six Spark Exercises to Rule Them All: https://towardsdatascience.com/six-spark-exercises-to-rule-them-all-242445b24565

* Managing "Exploding" Big Data https://engineering.linkedin.com/blog/2017/06/managing--exploding--big-data